In [14]:
#split train and test data
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os

cwd = os.getcwd()
df = pd.read_csv(cwd + "/data_csv/preprocessing_data.csv")

train, test = train_test_split(df, test_size=0.1)


In [15]:
train = pd.DataFrame(train, columns=["image", "emotion"])
dictionary = {'emotion':{'HAPPINESS': 0,'NEUTRAL': 1, 'SURPRISE': 2, 'ANGER': 3, 'SADNESS': 4, 'DISGUST': 5}}
train.replace(dictionary, inplace = True)
train.to_csv(cwd + "/data_csv/train.csv")

train.groupby('emotion').count()

,image
emotion,
0,2122
1,2013
2,76
3,74
4,30
5,5


In [16]:
test = pd.DataFrame(test, columns=["image", "emotion"])
dictionary = {'emotion':{'HAPPINESS': 0,'NEUTRAL': 1, 'SURPRISE': 2, 'ANGER': 3, 'SADNESS': 4, 'DISGUST': 5}}
test.replace(dictionary, inplace = True)
test.to_csv(cwd + "/data_csv/test.csv")

test.groupby('emotion').count()

,image
emotion,
0,237
1,223
2,10
3,4
4,6
5,1


In [17]:
'''
import cv2,matplotlib.pyplot as plt,dlib,imutils
from imutils import face_utils

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../input/dlib-68/shape_predictor_68_face_landmarks.dat")

image=plt.imread("../input/recognizing-faces-in-the-wild/train/F0002/MID1/P00009_face3.jpg")
# image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 1)

for rect in rects:
    pred=predictor(gray,rect)
    fig, ax1 = plt.subplots()

    ax1.imshow(image)
    ax1.scatter(face_utils.shape_to_np(pred)[:,0],face_utils.shape_to_np(pred)[:,1])
    
# del predictor
'''

'\nimport cv2,matplotlib.pyplot as plt,dlib,imutils\nfrom imutils import face_utils\n\ndetector = dlib.get_frontal_face_detector()\npredictor = dlib.shape_predictor("../input/dlib-68/shape_predictor_68_face_landmarks.dat")\n\nimage=plt.imread("../input/recognizing-faces-in-the-wild/train/F0002/MID1/P00009_face3.jpg")\n# image = imutils.resize(image, width=500)\ngray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\nrects = detector(gray, 1)\n\nfor rect in rects:\n    pred=predictor(gray,rect)\n    fig, ax1 = plt.subplots()\n\n    ax1.imshow(image)\n    ax1.scatter(face_utils.shape_to_np(pred)[:,0],face_utils.shape_to_np(pred)[:,1])\n    \n# del predictor\n'

In [18]:
'''
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

data = pd.read_csv(cwd + "/train_data.csv")

#independent columns
X = data.iloc[:,0:20]

#target column
y = data.iloc[:,-1]  

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns, dfscores],axis=1)

#naming the dataframe columns
featureScores.columns = ['Specs','Score']

#print 10 best features
print(featureScores.nlargest(10,'Score'))
'''

'\nfrom sklearn.feature_selection import SelectKBest\nfrom sklearn.feature_selection import chi2\n\ndata = pd.read_csv(cwd + "/train_data.csv")\n\n#independent columns\nX = data.iloc[:,0:20]\n\n#target column\ny = data.iloc[:,-1]  \n\n#apply SelectKBest class to extract top 10 best features\nbestfeatures = SelectKBest(score_func=chi2, k=10)\nfit = bestfeatures.fit(X,y)\ndfscores = pd.DataFrame(fit.scores_)\ndfcolumns = pd.DataFrame(X.columns)\n\n#concat two dataframes for better visualization \nfeatureScores = pd.concat([dfcolumns, dfscores],axis=1)\n\n#naming the dataframe columns\nfeatureScores.columns = [\'Specs\',\'Score\']\n\n#print 10 best features\nprint(featureScores.nlargest(10,\'Score\'))\n'

In [19]:
df = train

In [20]:
import cv2, math, numpy as np, dlib
from imutils import face_utils

# emotion list
emotions = [0, 1, 2, 3, 4, 5] 
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(cwd + "/predictor/shape_predictor_68_face_landmarks.dat")

error = 0

mlist = []
distlist = []
# run each image in train
for idx, row in df.iterrows():
    imagePath = cwd + "/cleaned_images/" + row.image
    image = cv2.imread(imagePath)

    rects = detector(image, 0)

    if len(rects) == 0:
        error += 1
        print(row.image)

    xlist = []
    ylist = []
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks and convert the facial landmark (x, y)
        shape = predictor(image, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over coordinates, draw them on the image and store coordinates in two lists
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
            cv2.imshow("Output", image)
            xlist.append(x)
            ylist.append(y)

    # get mean of both axes to determine centre of gravity
    xmean = np.mean(xlist) 
    ymean = np.mean(ylist)

    # plot central face on image
    cv2.circle(image, (int(xmean), int(ymean)), 1, (0, 255, 0), -1)

    # find distance between mouth
    mavg = np.mean([ylist[61] - ylist[67], ylist[62] - ylist[66], ylist[63] - ylist[65]])
    mlist.append(mavg)

    # find distance between every poin to central point
    templist = []
    for i in range(18, 68):
        dist = math.sqrt(math.pow(xlist[i] - xmean, 2) + math.pow(ylist[i] - ymean, 2))
        templist.append(dist)
    distavg = np.mean(dist)
    distlist.append(distavg)
  
    # show the output image with the face detections + facial landmarks
    cv2.imshow("Output", image)
    k = cv2.waitKey(5) & 0xFF
    if k == 68:
        break

print(error)

0


In [21]:
df['mouth_distance'] = mlist
df['average_distance'] = distlist

df.to_csv(cwd + "/data_csv/train.csv")

In [22]:
import numpy as np
import pandas as p
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

X = df[df.columns.difference(['emotion', 'image'])]
y = df.loc[:,'emotion']

print(X.shape, y.shape)

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.1, random_state=0)

print(X_train.shape, y_train.shape)
print(X_validate.shape, y_validate.shape)


clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
print(clf.score(X_validate, y_validate))

clf = svm.SVC(kernel='poly', C=1).fit(X_train, y_train)
print(clf.score(X_validate, y_validate))

clf = svm.SVC(kernel='rbf', C=1).fit(X_train, y_train)
print(clf.score(X_validate, y_validate))

(4320, 2) (4320,)
(3888, 2) (3888,)
(432, 2) (432,)
0.6736111111111112
0.6550925925925926
0.6643518518518519
